In [1]:
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mar_div_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/marriage_divorce_data.csv - Data.csv")
mar_div_df.fillna(value=0, inplace=True)
mar_div_df.head(10)

,co_fips,county,state,divorces_2010,marriages_2010
0,1001,Autauga,AL,231.0,341.0
1,1003,Baldwin,AL,980.0,2716.0
2,1005,Barbour,AL,69.0,228.0
3,1007,Bibb,AL,53.0,273.0
4,1009,Blount,AL,258.0,421.0
5,1011,Bullock,AL,7.0,76.0
6,1013,Butler,AL,83.0,184.0
7,1015,Calhoun,AL,682.0,899.0
8,1017,Chambers,AL,137.0,253.0
9,1019,Cherokee,AL,151.0,191.0


In [ ]:
# merge bedford CY, VA into Bedford County due to census data discrepency of identical nature

# bedford_cy_data = mar_div_df.keys()
bedford_cy_data = mar_div_df.loc[mar_div_df["county"] == 'Bedford CY']
bedf_cy_index = mar_div_df.loc[mar_div_df["county"] == 'Bedford CY'].index[0]
# mar_div_df.drop(mar_div_df.loc[mar_div_df["county"] == 'Bedford CY'], inplace=True)
divorce_value = mar_div_df.loc[mar_div_df["co_fips"] == 51019].divorces_2010.values + bedford_cy_data.divorces_2010.values[0]
bedf_co_index = mar_div_df.loc[mar_div_df["co_fips"] == 51019].index[0]
marriage_value = mar_div_df.loc[mar_div_df["co_fips"] == 51019].marriages_2010.values + bedford_cy_data.marriages_2010.values[0]
# mar_div_df.drop(mar_div_df.loc[mar_div_df["co_fips"] == 51019], inplace=True)
merge_data = pd.Series(data={"co_fips": 51019, "county": "Bedford", "state": "VA", "divorces_2010": divorce_value[0], "marriages_2010": marriage_value[0]})
mar_div_df.drop(index=[bedf_cy_index,bedf_co_index], inplace=True)
mar_div_df = mar_div_df.append(merge_data, ignore_index=True)
mar_div_df.tail(10)

,co_fips,county,state,divorces_2010,marriages_2010
3132,56029,Park,WY,137.0,223.0
3133,56031,Platte,WY,43.0,71.0
3134,56033,Sheridan,WY,157.0,252.0
3135,56035,Sublette,WY,60.0,89.0
3136,56037,Sweetwater,WY,224.0,257.0
3137,56039,Teton,WY,79.0,472.0
3138,56041,Uinta,WY,128.0,126.0
3139,56043,Washakie,WY,43.0,67.0
3140,56045,Weston,WY,31.0,42.0
3141,51019,Bedford,VA,234.0,413.0


In [ ]:
mar_div_df['county'] = mar_div_df['county'].str.replace(" ", "")
mar_div_df['county'] = mar_div_df['county'].str.replace(".", "")
mar_div_df['county'] = mar_div_df['county'].str.replace("CY", "city")
mar_div_df['Geographic Area'] = mar_div_df['county'].map(str) + ", " + mar_div_df['state']
mar_div_df.drop(columns=['county', 'state'], inplace=True)
mar_div_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


,co_fips,divorces_2010,marriages_2010,Geographic Area
0,1001,231.0,341.0,"Autauga, AL"
1,1003,980.0,2716.0,"Baldwin, AL"
2,1005,69.0,228.0,"Barbour, AL"
3,1007,53.0,273.0,"Bibb, AL"
4,1009,258.0,421.0,"Blount, AL"
...,...,...,...,...
3137,56039,79.0,472.0,"Teton, WY"
3138,56041,128.0,126.0,"Uinta, WY"
3139,56043,43.0,67.0,"Washakie, WY"
3140,56045,31.0,42.0,"Weston, WY"


In [ ]:
census_data_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Census_data_2010-2019.csv - CO-EST2019-ANNRES.csv')
census_data_df.head(10)

,Geographic Area,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,"308,745,538","308,758,105","309,321,666","311,556,874","313,830,990","315,993,715","318,301,008","320,635,163","322,941,311","324,985,539","326,687,501","328,239,523"
1,".Autauga County, Alabama","54,571","54,597","54,773","55,227","54,954","54,727","54,893","54,864","55,243","55,390","55,533","55,869"
2,".Baldwin County, Alabama","182,265","182,265","183,112","186,558","190,145","194,885","199,183","202,939","207,601","212,521","217,855","223,234"
3,".Barbour County, Alabama","27,457","27,455","27,327","27,341","27,169","26,937","26,755","26,283","25,806","25,157","24,872","24,686"
4,".Bibb County, Alabama","22,915","22,915","22,870","22,745","22,667","22,521","22,553","22,566","22,586","22,550","22,367","22,394"
5,".Blount County, Alabama","57,322","57,322","57,376","57,560","57,580","57,619","57,526","57,526","57,494","57,787","57,771","57,826"
6,".Bullock County, Alabama","10,914","10,911","10,876","10,675","10,606","10,549","10,663","10,400","10,389","10,176","10,174","10,101"
7,".Butler County, Alabama","20,947","20,940","20,932","20,866","20,670","20,356","20,327","20,162","20,012","19,888","19,631","19,448"
8,".Calhoun County, Alabama","118,572","118,526","118,408","117,744","117,190","116,471","115,917","115,469","114,973","114,710","114,331","113,605"
9,".Chambers County, Alabama","34,215","34,169","34,122","34,033","34,104","34,139","33,977","33,996","33,745","33,707","33,600","33,254"


In [ ]:
census_data_df.drop(columns=[ 'Estimates Base', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'], inplace=True)
census_data_df.drop(index = 0, inplace=True)

census_data_df.head(10)

,Geographic Area,Census
1,".Autauga County, Alabama","54,571"
2,".Baldwin County, Alabama","182,265"
3,".Barbour County, Alabama","27,457"
4,".Bibb County, Alabama","22,915"
5,".Blount County, Alabama","57,322"
6,".Bullock County, Alabama","10,914"
7,".Butler County, Alabama","20,947"
8,".Calhoun County, Alabama","118,572"
9,".Chambers County, Alabama","34,215"
10,".Cherokee County, Alabama","25,989"


In [ ]:
# change county names to split out state and create new comlumn with 2-letter code
additional_columns = pd.DataFrame(census_data_df["Geographic Area"].str.split(",",n=1, expand=True))
additional_columns = additional_columns.rename(columns={0:'county', 1:'state'})
additional_columns['county'] = additional_columns['county'].str.replace(".","")
additional_columns['county'] = additional_columns['county'].str.replace("County","",1)
additional_columns['county'] = additional_columns['county'].str.replace(" ","")
additional_columns['state'] = pd.Series(additional_columns['state'].str.replace(" ","").map(
    {'Alabama' : 'AL',
     'Arkansas' : 'AR',
     'Arizona' : 'AZ',
     'Alaska' : 'AK',
     'California' : 'CA',
     'Colorado' : 'CO',
     'Connecticut' : 'CT',
     'Delaware' : 'DE',
     'DistrictofColumbia' : 'DC',
     'Florida' : 'FL',
     'Georgia' : 'GA',
     'Hawaii' : 'HI',
     'Idaho' : 'ID',
     'Illinois' : 'IL',
     'Indiana' : 'IN',
     'Iowa' : 'IA',
     'Kansas' : 'KS',
     'Kentucky' : 'KY',
     'Louisiana' : 'LA',
     'Maine' : 'ME',
     'Maryland' : 'MD',
     'Massachusetts' : 'MA',
     'Michigan' : 'MI',
     'Minnisota' : 'MN',
     'Mississippi' : 'MS',
     'Missouri' : 'MO',
     'Montana' : 'MT',
     'Nebraska' : 'NE',
     'Nevada' : 'NV',
     'NewHampshire' : 'NH',
     'NewJersey' : 'NJ',
     'NewMexico' : 'NM',
     'NewYork' : 'NY',
     'NorthCarolina' : 'NC',
     'NorthDakota' : 'ND',
     'Ohio' : 'OH',
     'Oklahoma' : 'OK',
     'Oregon' : 'OR',
     'Pennsylvania' : 'PA',
     'RhodeIsland' : 'RI',
     'SouthCarolina' : 'SC',
     'SouthDakota' : 'SD',
     'Tennessee' : 'TN',
     'Texas' : 'TX',
     'Utah' : 'UT',
     'Vermont' : 'VT',
     'Virginia' : 'VA',
     'Washington' : 'WA',
     'WestVirginia' : 'WV',
     'Wisconsin' : 'WI',
     'Wyoming' : 'WY'}
))
# print(additional_columns['state abbr'].value_counts())
additional_columns['county_state_id'] = additional_columns['county'].map(str) + ", " + additional_columns['state']
census_data_df['Geographic Area'] = additional_columns['county_state_id']
census_data_df.head(10)

,Geographic Area,Census
1,"Autauga, AL","54,571"
2,"Baldwin, AL","182,265"
3,"Barbour, AL","27,457"
4,"Bibb, AL","22,915"
5,"Blount, AL","57,322"
6,"Bullock, AL","10,914"
7,"Butler, AL","20,947"
8,"Calhoun, AL","118,572"
9,"Chambers, AL","34,215"
10,"Cherokee, AL","25,989"


In [ ]:
census_data_df.dtypes

Geographic Area    object
Census             object
dtype: object

In [ ]:
# join the dataframes on 'Geographic Area' to merge the datasets
complete_county_mar_div_df = mar_div_df.merge(census_data_df, left_on='Geographic Area', right_on='Geographic Area', how='outer')
complete_county_mar_div_df.tail(158)

,co_fips,divorces_2010,marriages_2010,Geographic Area,Census
3142,NaN,NaN,NaN,"KusilvakCensusArea, AK","7,459"
3143,NaN,NaN,NaN,"PetersburgBorough, AK","3,815"
3144,NaN,NaN,NaN,"AcadiaParish, LA","61,773"
3145,NaN,NaN,NaN,"AllenParish, LA","25,764"
3146,NaN,NaN,NaN,"AscensionParish, LA","107,215"
...,...,...,...,...,...
3295,NaN,NaN,NaN,NaN,"10,438"
3296,NaN,NaN,NaN,"DoñaAna, NM","209,233"
3297,NaN,NaN,NaN,"Bristol, RI","49,875"
3298,NaN,NaN,NaN,"OglalaLakota, SD","13,586"


In [ ]:
complete_county_mar_div_df.loc[complete_county_mar_div_df["co_fips"] == 51019]

,co_fips,divorces_2010,marriages_2010,Geographic Area,Census
3141,51019.0,234.0,413.0,"Bedford, VA","68,676"
